In [827]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function


import datetime
#imports

# Ideas

Raise error when len does not match before and after merging datasets (means that a player wasn't found -- likely because he is a junior or has two last names, etc.)

# To Do

1. Address last name issue
2. Address issue of list not being equal to 30
    - Really need to add TBD or something for the pitchers who are not listed yet
3. Get pitcher xba onto
4. Deal with doubleheaders

In [828]:
batters_df = pd.read_csv('../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../data/individual_pitching_stats.csv')
misc_batters = pd.read_csv('../data/batters_teams.csv')
misc_pitchers = pd.read_csv('../data/pitchers_teams.csv')

In [829]:
duplicate_name_check = pitchers_df[['first_name', 'last_name']]

In [830]:
batters_df[batters_df['last_name'].str.contains(" ")]

,last_name,first_name,player_id,year,b_bb_percent,batting_avg,xba,xslg,woba,xwoba,...,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 22
5,Cruz Jr.,Nelson,443558,2022,11.6,0.160,0.262,0.484,0.234,0.354,...,0.399,0.203,0.334,-0.102,-0.244,-0.120,89.5,8.6,10.2,NaN
18,Brantley Jr.,Michael,488726,2022,7.9,0.290,0.330,0.563,0.341,0.396,...,0.446,0.345,0.393,-0.040,-0.143,-0.055,89.2,11.8,8.5,NaN
60,Springer III,George,543807,2022,5.0,0.288,0.266,0.452,0.379,0.329,...,0.381,0.362,0.336,0.022,0.069,0.050,90.5,17.6,5.2,NaN
87,Voit III,Luke,572228,2022,18.5,0.143,0.150,0.271,0.250,0.280,...,0.287,0.231,0.244,-0.007,-0.104,-0.030,85.9,19.0,7.7,NaN
129,Bradley Jr.,Jackie,598265,2022,10.9,0.161,0.219,0.349,0.232,0.290,...,0.319,0.214,0.292,-0.058,-0.117,-0.058,89.0,8.5,2.3,NaN
207,Mancini III,Trey,641820,2022,7.5,0.236,0.317,0.562,0.280,0.388,...,0.500,0.315,0.430,-0.081,-0.229,-0.108,91.6,9.8,11.1,NaN
243,Mullins II,Cedric,656775,2022,8.2,0.205,0.257,0.489,0.307,0.363,...,0.405,0.268,0.335,-0.052,-0.133,-0.056,89.2,18.8,12.3,NaN
282,Guerrero Jr.,Vladimir,665489,2022,11.4,0.309,0.329,0.672,0.402,0.438,...,0.526,0.382,0.414,-0.020,-0.113,-0.036,91.5,9.4,14.5,NaN
285,Chisholm Jr.,Jazz,665862,2022,8.5,0.308,0.268,0.561,0.436,0.383,...,0.506,0.421,0.397,0.040,0.112,0.053,89.1,14.8,15.8,NaN
295,Gurriel Jr.,Lourdes,666971,2022,3.8,0.274,0.271,0.461,0.322,0.329,...,0.386,0.333,0.341,0.003,-0.036,-0.007,92.3,13.4,5.0,NaN


In [831]:
len(batters_df)

335

In [832]:
bat_xbadiff = batters_df[['last_name', 'first_name', 'xbadiff','batting_avg', 'xba']]
#Make a table with just the xBA difference

In [833]:
pitch_xbadiff = pitchers_df[['last_name', 'first_name', 'xbadiff', 'batting_avg', 'xba']]
# pitch_xbadiff.sort_values('xba', ascending=False)

# Probable Pitchers

In [834]:
url = 'https://www.mlb.com/probable-pitchers/2022-05-06'
#URL of where I will get the pitchers' names

In [835]:
res = requests.get(url)
soup = BeautifulSoup(res.content)
#Making request to the site and storing it in soup

In [836]:
# pitchers = soup.find_all('div', {'class': 'probable-pitchers__pitcher-name'})

In [837]:
pitchers = soup.find_all('a', {'class': 'probable-pitchers__pitcher-name-link'})
probable_pitchers = []
for i in pitchers:
    probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

#Important to note that if a pitcher is TBD then he is skipped, so need to figure out how to account for that

In [838]:
probable_pitchers = [i.split('">')[1] for i in probable_pitchers] 
#Getting only the pitchers' names

In [839]:
probable_pitchers

['JT Brubaker</a>',
 'Connor Overton</a>',
 'Vince Velasquez</a>',
 'Nathan Eovaldi</a>',
 'Eric Lauer</a>',
 'Jesse Chavez</a>',
 'Beau Brieske</a>',
 'Luis Garcia</a>',
 'Cole Irvin</a>',
 'Josh Winder</a>',
 'Joan Adon</a>',
 'Chad Kuhl</a>',
 'Merrill Kelly</a>',
 'Matt Wisler</a>',
 'Logan Gilbert</a>',
 'Sandy Alcantara</a>',
 'Yu Darvish</a>',
 'Jordan Hicks</a>',
 'Alex Cobb</a>',
 'Tyler Anderson</a>',
 'Carlos Hernandez</a>',
 'Jordan Lyles</a>',
 'Max Scherzer</a>',
 'Kyle Gibson</a>',
 'Glenn Otto</a>',
 'Gerrit Cole</a>',
 'Kevin Gausman</a>',
 'Shane Bieber</a>']

In [840]:
probable_pitchers = [i.split('</a')[0] for i in probable_pitchers] 
#Getting only the pitchers' names part 2

In [841]:
probable_pitchers = [i.split() for i in probable_pitchers]
#Splitting name into first and last name columns

In [842]:
len(probable_pitchers)

28

In [843]:
probable_pitchers.insert(23, ['Drew', 'Smyly'])
probable_pitchers.insert(9, ['Shohei', 'Ohtani'])
#Temporary

In [844]:
len(probable_pitchers)

30

In [845]:
df_probable_pitchers = pd.DataFrame(probable_pitchers)
#Setting list as Pandas DF

In [846]:
df_probable_pitchers = df_probable_pitchers.rename(columns={0: "first_name", 1: "last_name"})
#Renaming column

# Get Pitchers' Teams

In [847]:
new_teams = soup.find_all('span', {'class':['probable-pitchers__team-name probable-pitchers__team-name--away', 'probable-pitchers__team-name probable-pitchers__team-name--home']})
teams_of_probable_pitchers = []
for i in new_teams:
    teams_of_probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [848]:
teams_of_probable_pitchers = [x.strip(' ') for x in teams_of_probable_pitchers]
#Getting only the pitchers' names

In [849]:
teams_of_probable_pitchers = [i.split('>\n')[1] for i in teams_of_probable_pitchers] 
#Getting only the pitchers' names

In [850]:
teams_of_probable_pitchers = teams_of_probable_pitchers = [x.strip('                             ') for x in teams_of_probable_pitchers]
#Stripping white space

In [851]:
teams_of_probable_pitchers = [i.split('\n')[0] for i in teams_of_probable_pitchers] 

In [852]:
teams_of_probable_pitchers

['Pirates',
 'Reds',
 'White Sox',
 'Red Sox',
 'Brewers',
 'Braves',
 'Tigers',
 'Astros',
 'Athletics',
 'Twins',
 'Nationals',
 'Angels',
 'Rockies',
 'D-backs',
 'Rays',
 'Mariners',
 'Marlins',
 'Padres',
 'Cardinals',
 'Giants',
 'Dodgers',
 'Cubs',
 'Royals',
 'Orioles',
 'Mets',
 'Phillies',
 'Rangers',
 'Yankees',
 'Blue Jays',
 'Guardians']

In [853]:
df_probable_pitchers

,first_name,last_name
0,JT,Brubaker
1,Connor,Overton
2,Vince,Velasquez
3,Nathan,Eovaldi
4,Eric,Lauer
5,Jesse,Chavez
6,Beau,Brieske
7,Luis,Garcia
8,Cole,Irvin
9,Shohei,Ohtani


In [854]:
df_probable_pitchers['team'] = teams_of_probable_pitchers
#Putting team on this way, now we just need to change it to the abbreviation

In [855]:
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Astros', 'HOU', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Nationals', 'WAS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Giants', 'SF', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Blue Jays', 'TOR', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Angels', 'LAA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'White Sox', 'CWS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Guardians', 'CLE', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Athletics', 'OAK', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Twins', 'MIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rays', 'TB', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mariners', 'SEA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Marlins', 'MIA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Padres', 'SD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Pirates', 'PIT', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Red Sox', 'BOS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Orioles', 'BAL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Braves', 'ATL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rangers', 'TEX', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Phillies', 'PHI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mets', 'NYM', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Yankees', 'NYY', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Royals', 'KC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cubs', 'CHC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Brewers', 'MIL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Reds', 'CIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rockies', 'COL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Tigers', 'DET', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Dodgers', 'LAD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'D-backs', 'ARI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cardinals', 'STL', df_probable_pitchers['team'])
#Changing team nickname to abbreviation so it will match the other DF -- Will clean this up in the future

In [856]:
misc_pitchers = misc_pitchers[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [857]:
misc_pitchers[['first_name', 'last_name']] = misc_pitchers['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [858]:
misc_pitchers = misc_pitchers.drop(columns='Player')
#We don't need this full name column anymore

In [859]:
misc_pitchers = misc_pitchers.rename(columns={"Team": "team"})
#Make this lowercase so it matches team column in probable pitchers

In [860]:
misc_pitchers = misc_pitchers[['last_name', 'first_name', 'team']]
#reordering columns

# Get Batters' Teams

In [861]:
misc_batters = misc_batters[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [862]:
misc_batters[['first_name', 'last_name']] = misc_batters['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [863]:
misc_batters = misc_batters.drop(columns='Player')
#We don't need this full name column anymore

In [864]:
misc_batters = misc_batters[['last_name', 'first_name', 'Team']]
#Reordering columns

In [865]:
len(df_probable_pitchers)

30

In [866]:
pitchers_df['first_name'] = pitchers_df['first_name'].str.strip()
batters_df['first_name'] = batters_df['first_name'].str.strip()
#Removing empty space in these column entries for example this makes ' Cabrera' => 'Cabrera'

batters_df and pitchers_df front names all have a space in front of them and that's why merge is not working

# Attaching according Team Name to each player in all DFs

The datasets on Baseball Savant do not include the team name with they players, so I will add them in

In [867]:
pitchers_df

,last_name,first_name,player_id,year,p_bb_percent,batting_avg,xba,xslg,woba,xwoba,...,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 22
0,Perez,Oliver,424144,2022,4.2,0.444,0.380,0.691,0.448,0.416,...,0.407,0.400,0.347,0.064,-0.024,0.032,88.8,16.0,10.0,NaN
1,Wainwright,Adam,425794,2022,7.7,0.313,0.301,0.507,0.365,0.369,...,0.464,0.419,0.407,0.012,-0.049,-0.004,87.0,9.1,9.7,NaN
2,Greinke,Zack,425844,2022,3.4,0.266,0.338,0.544,0.287,0.403,...,0.415,0.276,0.367,-0.072,-0.202,-0.116,89.6,11.7,5.3,NaN
3,Verlander,Justin,434378,2022,4.3,0.161,0.215,0.389,0.219,0.275,...,0.369,0.233,0.313,-0.054,-0.113,-0.056,88.1,14.5,8.2,NaN
4,Jansen,Kenley,445276,2022,6.9,0.111,0.140,0.238,0.141,0.197,...,0.270,0.188,0.236,-0.029,-0.127,-0.056,85.2,25.4,6.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,Roberts,Ethan,681799,2022,14.3,0.310,0.312,0.612,0.471,0.439,...,0.558,0.450,0.452,-0.002,0.043,0.032,88.8,14.0,15.0,NaN
488,Warren,Austin,681810,2022,16.1,0.333,0.250,0.414,0.421,0.342,...,0.356,0.381,0.305,0.083,0.169,0.079,90.0,14.6,4.5,NaN
489,Vesia,Alex,681911,2022,4.8,0.200,0.279,0.563,0.224,0.367,...,0.501,0.286,0.399,-0.079,-0.313,-0.143,90.2,17.4,14.3,NaN
490,Brieske,Beau,689225,2022,10.0,0.167,0.295,0.532,0.332,0.381,...,0.415,0.200,0.354,-0.128,-0.032,-0.049,88.2,10.2,13.3,NaN


In [868]:
df_probable_pitchers = pd.merge(df_probable_pitchers, misc_pitchers, on=['last_name', 'first_name', 'team'], how='left')
pitchers_df = pd.merge(pitchers_df, misc_pitchers, on=['last_name', 'first_name'], how='inner')
batters_df = pd.merge(batters_df, misc_batters, on=['last_name', 'first_name'], how='inner')
#Adding team to all of the players
#Joining probables as a left join so it will include all of the probables without exception -- may make one of their stats
# unavailable if they don't have enough PAs 

In [869]:
# ^^^^ FOR PROBABLE PITCHERS MATCH IT ON TEAM AS WELL. THAT WILL AVOID THE LUIS GARCIA SITUATION

In [870]:
pitchers_df.loc[pitchers_df['last_name'] == 'Garcia']
#So somewhere it has duplicated Garcia

,last_name,first_name,player_id,year,p_bb_percent,batting_avg,xba,xslg,woba,xwoba,...,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 22,team
22,Garcia,Luis,472610,2022,0.0,0.182,0.309,0.405,0.164,0.309,...,0.235,0.400,-0.127,-0.223,-0.145,93.2,0.7,5.9,NaN,HOU
23,Garcia,Luis,472610,2022,0.0,0.182,0.309,0.405,0.164,0.309,...,0.235,0.400,-0.127,-0.223,-0.145,93.2,0.7,5.9,NaN,SD
24,Garcia,Luis,677651,2022,6.6,0.196,0.204,0.440,0.297,0.290,...,0.256,0.265,-0.008,-0.011,0.007,88.6,24.8,14.0,NaN,HOU
25,Garcia,Luis,677651,2022,6.6,0.196,0.204,0.440,0.297,0.290,...,0.256,0.265,-0.008,-0.011,0.007,88.6,24.8,14.0,NaN,SD
89,Garcia,Yimi,554340,2022,9.1,0.214,0.235,0.329,0.264,0.269,...,0.240,0.265,-0.021,-0.008,-0.005,88.4,11.8,4.0,NaN,TOR
183,Garcia,Jarlin,606424,2022,25.0,0.067,0.277,0.527,0.219,0.426,...,0.100,0.415,-0.210,-0.460,-0.207,95.4,11.2,20.0,NaN,SF
415,Garcia,Rony,665621,2022,8.0,0.217,0.291,0.657,0.252,0.414,...,0.278,0.371,-0.074,-0.396,-0.162,89.1,19.7,16.7,NaN,DET


In [871]:
hello = pd.merge(df_probable_pitchers, pitchers_df)

In [872]:
df_probable_pitchers = pd.merge(df_probable_pitchers, pitchers_df, on=['team', 'last_name', 'first_name' ], how='left')
#Adding stats to probable pitchers

#This is what is adding the other Luis Garcia

# Which player to pick for BTS

In [873]:
pp_xba = df_probable_pitchers[['last_name', 'first_name', 'team', 'batting_avg', 'xba', 'p_bb_percent']]
#How can I compare this to batters df??

In [874]:
pp_xba.rename(columns= {'xba': 'pitcher_xba'})
#Eventually might move this to another df or merge, unclear

,last_name,first_name,team,batting_avg,pitcher_xba,p_bb_percent
0,Brubaker,JT,PIT,0.226,0.224,15.1
1,Overton,Connor,CIN,NaN,NaN,NaN
2,Velasquez,Vince,CWS,0.288,0.312,11.9
3,Eovaldi,Nathan,BOS,0.247,0.271,3.4
4,Lauer,Eric,MIL,0.210,0.238,6.0
5,Chavez,Jesse,ATL,0.306,0.317,5.3
6,Brieske,Beau,DET,0.167,0.295,10.0
7,Garcia,Luis,HOU,0.182,0.309,0.0
8,Garcia,Luis,HOU,0.196,0.204,6.6
9,Irvin,Cole,OAK,0.226,0.296,5.6


One thing I could do is... switch the one column with the next, and then merge it with the batter's df
So for example if Oakland is playing Cleveland, I can switch the pitchers with each other. So Oakland's pitcher will be on Cleveland, and then I will have the pitcher xba to match with the team. Then I can basically set that for every member of the team, and then add that with the hitter xba and see if it meets the condition

In [875]:
pp_xba.sort_values(by='xba', ascending=False)

,last_name,first_name,team,batting_avg,xba,p_bb_percent
5,Chavez,Jesse,ATL,0.306,0.317,5.3
2,Velasquez,Vince,CWS,0.288,0.312,11.9
7,Garcia,Luis,HOU,0.182,0.309,0.0
9,Irvin,Cole,OAK,0.226,0.296,5.6
6,Brieske,Beau,DET,0.167,0.295,10.0
29,Gausman,Kevin,TOR,0.266,0.281,0.0
3,Eovaldi,Nathan,BOS,0.247,0.271,3.4
14,Kelly,Merrill,ARI,0.244,0.259,7.9
16,Gilbert,Logan,SEA,0.188,0.246,4.7
30,Bieber,Shane,CLE,0.188,0.245,5.8


In [876]:
batters_df.loc[batters_df['Team'] == 'CWS']

,last_name,first_name,player_id,year,b_bb_percent,batting_avg,xba,xslg,woba,xwoba,...,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 22,Team
30,Grandal,Yasmani,518735,2022,8.6,0.173,0.250,0.506,0.236,0.346,...,0.214,0.315,-0.077,-0.237,-0.110,87.5,12.6,14.3,NaN,CWS
49,Harrison,Josh,543281,2022,2.9,0.094,0.214,0.414,0.156,0.289,...,0.120,0.273,-0.120,-0.226,-0.133,92.1,20.3,8.0,NaN,CWS
61,Garcia,Leury,544725,2022,4.3,0.114,0.211,0.327,0.165,0.249,...,0.147,0.273,-0.097,-0.122,-0.084,88.1,6.1,2.9,NaN,CWS
68,Abreu,Jose,547989,2022,10.0,0.206,0.256,0.576,0.291,0.378,...,0.277,0.343,-0.050,-0.227,-0.087,93.8,8.6,14.9,NaN,CWS
181,McGuire,Reese,624512,2022,6.1,0.167,0.253,0.449,0.191,0.319,...,0.217,0.340,-0.086,-0.249,-0.128,86.4,8.1,8.7,NaN,CWS
189,Anderson,Tim,641313,2022,1.6,0.288,0.310,0.580,0.343,0.382,...,0.347,0.374,-0.022,-0.122,-0.039,89.5,9.0,14.3,NaN,CWS
195,Engel,Adam,641553,2022,11.1,0.188,0.246,0.319,0.239,0.292,...,0.273,0.357,-0.058,-0.100,-0.053,89.5,8.0,0.0,NaN,CWS
228,Jimenez,Eloy,650391,2022,5.1,0.222,0.234,0.418,0.264,0.288,...,0.296,0.313,-0.012,-0.085,-0.024,92.6,3.0,7.4,NaN,CWS
242,Sheets,Gavin,657757,2022,4.3,0.227,0.258,0.423,0.273,0.314,...,0.303,0.344,-0.031,-0.105,-0.041,89.5,15.3,9.1,NaN,CWS
297,Burger,Jake,669394,2022,2.0,0.277,0.256,0.518,0.329,0.328,...,0.382,0.354,0.021,-0.071,0.001,90.7,18.2,11.4,NaN,CWS


In [877]:
pp_xba['Team'] = " "
#New column to house when I switch the teams with each other
#I'm doing this so I can eventually merge it onto the batters' df with the hitters' team

C:\Users\User\AppData\Local\Temp/ipykernel_6268/1964202557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'] = " "


In [878]:
# pp_xba['opposing_team'][0] = pp_xba['team'][1]
# pp_xba['']
#Open to a cleaner way of doing this, but this is what I'll use for now

# Getting Pitcher Stats onto Batter DF

FOR NOW, UNTIL I CLEAN THE BELOW UP A LITTLE MORE AND HAVE IT ITERATE THROUGH AND ADJUST TO THE NUMBER OF ROWS PER DAY, JUST USE THE AMOUNT OF ROWS THAT THE LEN() SHOWS AND COMMENT OUT THE REST

In [879]:
len(pp_xba)

31

In [880]:
pp_xba['Team'][0] = pp_xba['team'][1]
pp_xba['Team'][1] = pp_xba['team'][0]
pp_xba['Team'][2] = pp_xba['team'][3]
pp_xba['Team'][3] = pp_xba['team'][2]
pp_xba['Team'][4] = pp_xba['team'][5]
pp_xba['Team'][5] = pp_xba['team'][4]
pp_xba['Team'][6] = pp_xba['team'][7]
pp_xba['Team'][7] = pp_xba['team'][6]
pp_xba['Team'][8] = pp_xba['team'][9]
pp_xba['Team'][9] = pp_xba['team'][8]
pp_xba['Team'][10] = pp_xba['team'][11]
pp_xba['Team'][11] = pp_xba['team'][10]
pp_xba['Team'][12] = pp_xba['team'][13]
pp_xba['Team'][13] = pp_xba['team'][12]
pp_xba['Team'][14] = pp_xba['team'][15]
pp_xba['Team'][15] = pp_xba['team'][14]
pp_xba['Team'][16] = pp_xba['team'][17]
pp_xba['Team'][17] = pp_xba['team'][16]
pp_xba['Team'][18] = pp_xba['team'][19]
pp_xba['Team'][19] = pp_xba['team'][18]
pp_xba['Team'][20] = pp_xba['team'][21]
pp_xba['Team'][21] = pp_xba['team'][20]
pp_xba['Team'][22] = pp_xba['team'][23]
pp_xba['Team'][23] = pp_xba['team'][22]
pp_xba['Team'][24] = pp_xba['team'][25]
pp_xba['Team'][25] = pp_xba['team'][24]
pp_xba['Team'][26] = pp_xba['team'][27]
pp_xba['Team'][27] = pp_xba['team'][26]
pp_xba['Team'][28] = pp_xba['team'][29]
pp_xba['Team'][29] = pp_xba['team'][28]
# pp_xba['Team'][30] = pp_xba['team'][31]
# pp_xba['Team'][31] = pp_xba['team'][30]

C:\Users\User\AppData\Local\Temp/ipykernel_6268/2011693318.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][0] = pp_xba['team'][1]
C:\Users\User\AppData\Local\Temp/ipykernel_6268/2011693318.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][1] = pp_xba['team'][0]
C:\Users\User\AppData\Local\Temp/ipykernel_6268/2011693318.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp_xba['Team'][2] = pp_xba['te

In [881]:
pp_xba = pp_xba.rename(columns={"batting_avg": "pitcher_batting_avg", "xba": "pitcher_xba"})

In [882]:
pp_xba = pp_xba.drop([8])
#temporary

In [883]:
batters_df.head()

,last_name,first_name,player_id,year,b_bb_percent,batting_avg,xba,xslg,woba,xwoba,...,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,Unnamed: 22,Team
0,Pujols,Albert,405395,2022,6.7,0.259,0.259,0.481,0.379,0.348,...,0.333,0.333,0.000,0.038,0.031,90.4,19.3,4.8,NaN,STL
1,Cabrera,Miguel,408234,2022,6.7,0.296,0.235,0.330,0.306,0.288,...,0.421,0.356,0.061,0.003,0.018,91.5,5.4,7.9,NaN,DET
2,Molina,Yadier,425877,2022,0.0,0.200,0.191,0.240,0.190,0.187,...,0.222,0.212,0.009,-0.015,0.003,85.9,15.9,0.0,NaN,STL
3,Cano,Robinson,429664,2022,5.0,0.184,0.222,0.372,0.224,0.271,...,0.241,0.291,-0.038,-0.109,-0.047,85.2,4.4,6.9,NaN,NYM
4,Suzuki,Kurt,435559,2022,12.1,0.172,0.167,0.257,0.272,0.241,...,0.217,0.210,0.005,0.053,0.031,84.3,23.0,0.0,NaN,LAA


In [885]:
batters_df = pd.merge(batters_df, pp_xba[['Team','pitcher_xba']],on='Team', how='left')
#Getting pitcher data onto batters df

In [886]:
xba_sum = batters_df['xba'] + batters_df['pitcher_xba']

# Which Hitters to pick for BTS

Now we need to see 